In [39]:
from sklearn.datasets import load_digits 
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score 
import numpy as np
import tensorflow as tf

### Utility functions

#### Data collection and preprocessing

In [35]:
def convert_y_to_vect(y):
    y_vect = np.zeros((len(y), 10))
    for i in range(len(y)):
        y_vect[i, y[i]] = 1
    return y_vect


def load_mnist_data(source="sk"):
    if source == "sk":
        digits=load_digits()
        X = digits.data
        y = digits.target
    else:
        mnist = tf.keras.datasets.mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        X = np.append(x_train, x_test, axis=0)
        X = X.reshape(-1, X.shape[1]**2)
        y = np.append(y_train, y_test, axis=0)
        
    X_scale = StandardScaler()
    X = X_scale.fit_transform(X)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    y_train = convert_y_to_vect(y_train)
    
    return (x_train, y_train), (x_test, y_test)

### TF Implementation - Emulate Homework
We use TensorFlow to emulate our implementation for Neural Netork Assignment. We chose a rather low performing combination of our hyperparameters from homework implementation to test out the robustness and achieve higher accuracies. 
##### Hyperparameter combination
Activation - **Relu** || Weights and Biases initialization - **Uniform Distribution** || Optimizer - **SGD** || Alpha - **0.1** || Regularization - **L2 (0.01)** || Epochs - **300**


In [36]:
def generate_model(data_source="sk"):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(64 if data_source=="sk" else 784,)),
        tf.keras.layers.Dense(30, activation="relu", 
                              kernel_initializer="glorot_uniform", 
                              bias_initializer="glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(10, activation="relu",
                              kernel_initializer="glorot_uniform", 
                              bias_initializer="glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))
    ])
    return model


def tf_implementation(data_source="sk", no_runs=1, batch_size=32, epochs=300, verbose=0):
    avg_accuracy = []
    for i in range(1, no_runs+1):
        (x_train, y_train), (x_test, y_test) = load_mnist_data(source=data_source)
        model = generate_model(data_source)
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss='mse', metrics=['accuracy'])
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)
        preds = model.predict(x_test)
        y_preds = np.argmax(preds, axis=1).reshape(-1,1)
        accuracy = accuracy_score(y_test, y_preds)
        print("Tensor Flow Implementaion accuracy {:0.2f} %".format(accuracy*100))
        avg_accuracy.append(accuracy)
    if no_runs > 1:
        print("Average accuracy over {} runs = {:0.2f} %".format(no_runs, (np.sum(np.array(avg_accuracy))/float(no_runs))*100))

### TF Extension - RMS Prop

In [37]:
def generate_model_ext(data_source="sk"):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(64 if data_source=="sk" else 784,)),
        tf.keras.layers.Dense(30, activation="relu",
                              kernel_initializer="glorot_uniform",
                              bias_initializer="glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(10, activation="relu",
                              kernel_initializer="glorot_uniform", 
                              bias_initializer="glorot_uniform",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

    ])
    return model

def tf_ext(data_source="sk", lr=0.001, no_runs=1, batch_size=32, epochs=100, verbose=0):
    avg_accuracy = []
    for i in range(1, no_runs+1):
        (x_train, y_train), (x_test, y_test) = load_mnist_data(data_source)
        model = generate_model_ext(data_source)
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr, rho=0.9), loss='mse', metrics=['accuracy'])
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=verbose)
        preds = model.predict(x_test)
        y_preds = np.argmax(preds, axis=1).reshape(-1,1)
        accuracy = accuracy_score(y_test, y_preds)
        print("Tensor Flow extension using RMSprop {:0.2f} %".format(accuracy*100))
        avg_accuracy.append(accuracy)
    if no_runs > 1:
        print("Average accuracy over {} runs = {:0.2f} %".format(no_runs, (np.sum(np.array(avg_accuracy))/float(no_runs))*100))

## Testing

## Scikit-Learn MNIST (approx. 2000 images)

In [40]:
tf_implementation(data_source = "sk", no_runs=5, batch_size=32, epochs=300, verbose=0)

Tensor Flow Implementaion accuracy 85.81 %
Tensor Flow Implementaion accuracy 68.29 %
Tensor Flow Implementaion accuracy 76.77 %
Tensor Flow Implementaion accuracy 67.87 %
Tensor Flow Implementaion accuracy 86.79 %
Average accuracy over 5 runs = 77.11 %


In [46]:
tf_ext(data_source="sk", lr=0.001, no_runs=5, batch_size=32, epochs=300, verbose=0)

Tensor Flow extension using RMSprop 74.27 %
Tensor Flow extension using RMSprop 94.16 %
Tensor Flow extension using RMSprop 72.04 %
Tensor Flow extension using RMSprop 86.37 %
Tensor Flow extension using RMSprop 93.18 %
Average accuracy over 5 runs = 84.01 %


## MNIST Handwritten digits (approx. 70K images)

In [42]:
tf_implementation(data_source="tf", no_runs=5, batch_size=512, epochs=10, verbose=0)

Tensor Flow Implementaion accuracy 88.23 %
Tensor Flow Implementaion accuracy 72.68 %
Tensor Flow Implementaion accuracy 88.56 %
Tensor Flow Implementaion accuracy 89.21 %
Tensor Flow Implementaion accuracy 73.51 %
Average accuracy over 5 runs = 82.44 %


In [47]:
tf_ext(data_source="tf", lr=0.001, no_runs=5, batch_size=512, epochs=10, verbose=0)

Tensor Flow extension using RMSprop 86.03 %
Tensor Flow extension using RMSprop 87.37 %
Tensor Flow extension using RMSprop 71.08 %
Tensor Flow extension using RMSprop 87.80 %
Tensor Flow extension using RMSprop 87.80 %
Average accuracy over 5 runs = 84.01 %
